Importing Data

In [1]:
import rasterio
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
data=pd.read_excel('new_without B1 and B9.xlsx')
x=data.drop (columns=["Target"])
y=data ["Target"]
x=x.to_numpy ()
y=y.to_numpy ()
y=y.reshape (y.shape [0], 1)
x_train, x_test, y_train, y_test = train_test_split (x, y, test_size = 0.2, random_state =42)


Artificial Neural Network model (ANN)

In [2]:
from tensorflow import keras
from tensorflow.keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras.utils import plot_model
model = Sequential ()
model.add (Dense (14, input_shape=x_train.shape, activation= "relu"))
model.add (Dense (12, activation = 'relu'))
model.add (Dropout (0.5))
model.add (Dense (8, activation = 'relu'))
model.add (Dense (1, activation = 'sigmoid'))
# plot_model (model, show_shapes=True, show_layer_names= True)
model.compile (loss="binary_crossentropy", optimizer = "adam", metrics=['acc',f1_m,precision_m, recall_m])
model.fit (x_train, y_train, epochs=400, batch_size=32, verbose=0)
# recall, precision = model.evaluate(x_test, y_test, verbose=0)
loss, accuracy, f1_score, precision, recall = model.evaluate(x_test, y_test, verbose=0)








In [ ]:
Metrics 

In [3]:
print (loss)
print (accuracy)
print (f1_score)
print (precision)
print (recall)

0.1771978735923767
0.9271523356437683
0.7992077469825745
0.7719601988792419
0.8379846215248108


In [ ]:
Confusion matrix

In [91]:
# pred=model.predict (x_test)
# pred=K.round(K.clip(pred, 0, 1))
# confusion_matrix (pred, y_test)

array([[257,  16],
       [  3, 119]], dtype=int64)

In [ ]:
Apply for Imagery and Writing to Disk 

In [4]:
import numpy
data_1=rasterio.open ('1-6-2019.tif')
data_2=data_1.read ()
data_3=np.swapaxes (data_2, 0, 2)
data_3=np.swapaxes (data_3, 0, 1)
data_3 = data_3.reshape (data_3.shape [0]*data_3.shape [1], data_3.shape [2])
prediction = model.predict (data_3)
prediction=K.round(K.clip(prediction, 0, 1))
prediction= prediction.numpy ()
prediction=prediction.reshape (1, data_2.shape [1], data_2.shape [2])
with rasterio.open ("1-6-2019_ANN_2_new.tif", "w", 
              driver="GTiff",
              width=data_2.shape[2],
              height=data_2.shape [1],
              count=1,
              crs=data_1.crs,
              transform=data_1.transform,
              dtype=rasterio.float32) as dst:
    dst.write (prediction)